In [ ]:
import shutil, os, subprocess
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/models/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import kagglehub

path = kagglehub.dataset_download("shivamb/company-acquisitions-7-top-companies")

print("Path to dataset files:", path)

100%|██████████| 30.9k/30.9k [00:00<00:00, 5.99MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/shivamb/company-acquisitions-7-top-companies/versions/2


In [ ]:
import json
with open('myconfig.json') as data_file:
  myconfig = json.load(data_file)
print(myconfig.keys())

dict_keys(['OPEN_API_KEY', 'OPENAI_API_KEY', 'DB_HOST', 'DB_PORT', 'DB_USER', 'DB_PASSWORD', 'HF_TOKEN', 'CLIPDROP_KEY', 'MJ_APIKEY_PUBLIC', 'MJ_APIKEY_PRIVATE', 'MYEMAIL'])


In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-_7iyz358/unsloth_af4865a7f933469785e785f8636e05e2
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-_7iyz358/unsloth_af4865a7f933469785e785f8636e05e2
  Resolved https://github.com/unslothai/unsloth.git to commit fd753fed99ed5f10ef8a9b7139588d9de9ddecfb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth: Fast Llama patching release 2024.8
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
mna_news_input = """HCL Technologies has completed acquisition of 51% stake in German IT Consulting Company, Gesellschaft für Banksysteme GmbH (“GBS”). The acquisition was completed on January 05, 2022."""

mna_news_instruction1 = """What is the acquistion date. Answer precisely"""
mna_news_instruction2 = """Which company is the Acquirer. . Answer precisely """

In [ ]:
alpaca_prompt.format(mna_news_instruction1, mna_news_input, "")

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is the acquistion date. Answer precisely\n\n### Input:\nHCL Technologies has completed acquisition of 51% stake in German IT Consulting Company, Gesellschaft für Banksysteme GmbH (“GBS”). The acquisition was completed on January 05, 2022.\n\n### Response:\n'

In [ ]:
FastLanguageModel.for_inference(model)

In [ ]:
inputs = tokenizer([alpaca_prompt.format(mna_news_instruction1, mna_news_input,"", )], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

tokenizer.batch_decode(outputs)[0].split("\n\n### Response:\n")[1].split("\n\n### Explanation:\n")[0]

'January 05, 2022\n<|end_of_text|>'

In [ ]:
inputs = tokenizer([alpaca_prompt.format(mna_news_instruction2, mna_news_input, "", )], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

tokenizer.batch_decode(outputs)[0].split("\n\n### Response:\n")[1].split("\n\n### Explanation:\n")[0]

'HCL Technologies is the Acquirer.\n\n<|end_of_text|>'